In [1]:
from PIL import Image
import numpy as np
from tifffile import imread,imsave
from sklearn.metrics import jaccard_score
from glob import glob
import random
import os

In [2]:
def setdiff_sorted(array1,array2,assume_unique=False):
    ans = np.setdiff1d(array1,array2,assume_unique).tolist()
    if assume_unique:
        return sorted(ans)
    return ans

In [3]:
im_path = glob('/Users/prakash/Desktop/MetaSeg_Data_Version1/c.elegans/filtered_labels/label1_filtered/unsynched/*.tif')
goldGT_TRA_path = glob('/Users/prakash/Desktop/MetaSeg_Data_Version1/c.elegans/SEG/*.tif')
temp_path = '/Users/prakash/Desktop/MetaSeg_Data_Version1/c.elegans/filtered_labels/label1_filtered/unsynched/temp/'
out_path = '/Users/prakash/Desktop/MetaSeg_Data_Version1/c.elegans/filtered_labels/Sixteenbit_synched_label1/synched/'

In [4]:
for i in range(len(goldGT_TRA_path)):
    
    im_name = im_path[i]
    fp = open(im_name, "rb")
    im = Image.open(fp)  # open from file object
    im.load()  # make sure PIL has read the data
    im_seg = np.array(im)
    fp.close()  # close the file so that we can re-save it again
    goldGT_TRA_name = goldGT_TRA_path[i]
    tra_markers_im = imread(goldGT_TRA_name)    # load TRA markers. i.e. source for label-sync
    tra_markers = np.unique(tra_markers_im)[1:]     # excluding BG:0
    result_markers = np.unique(im_seg)[1:] # excluding BG:0
    temp_ids = np.arange(np.max(tra_markers),np.max(tra_markers)+len(result_markers),1)
 
    if len(temp_ids)<len(result_markers):
        print(tra_markers, result_markers)
        print(temp_ids)
        raise "Not enough temporary ids to relabel, exiting."
    
    temp_id_counter = 0
    seg_copy = np.zeros((im_seg.shape[0], im_seg.shape[1]))
    for cell_index in result_markers:
        seg_copy[im_seg == cell_index] = temp_ids[temp_id_counter]
        temp_id_counter+=1
    im = Image.fromarray(seg_copy)
    if not os.path.exists(temp_path):
        os.mkdir(temp_path)
    im.save(temp_path+"mask"+str(i).zfill(3)+".tif", dpi=(25, 25))   # Fix dpi value for all available results. STAPLE executable requires inputs with same dpi


In [5]:
for i in range(len(goldGT_TRA_path)):
    im_name = im_path[i]
    im_seg = np.array(imread(im_name))
    goldGT_TRA_name = goldGT_TRA_path[i]
    tra_markers_im = imread(goldGT_TRA_name)    # load TRA markers. i.e. source for label-sync
    tra_markers = np.unique(tra_markers_im)[1:]     # excluding BG:0
        
    im_out = np.copy(im_seg)
    for cell_index in tra_markers:
        im_tra = np.copy(tra_markers_im)
        im_tra[im_tra != cell_index] = 0  # keep only one cell to process each cell individually
        im_tra[im_tra == cell_index] = 1   # to keep values in the multiplication operation
        size_traGT = np.count_nonzero(im_tra)
        intersection = im_tra * im_seg
        size_intersect = np.count_nonzero(intersection)
        labels,count = np.unique(intersection, return_counts=True)
        count_sort_ind = np.argsort(-count)
        sorted_labels = labels[count_sort_ind]
        if(sorted_labels[0]==0 and len(sorted_labels)>1): #If background label has most counts, then consider the second most
            label_by_participant = sorted_labels[1]
        elif (sorted_labels[0]!=0):
            label_by_participant = sorted_labels[0]
        else: #If background is the only label in result for any GT object
            continue

        label_coordinates = np.where(im_seg == label_by_participant)
        if size_intersect >= (size_traGT/2):            # if the current marker covers at least half of the TRA marker
            im_out[label_coordinates] = cell_index      # sync! keep the current label but change its value to TRA GT
#         else:
#             im_out[label_coordinates] = 0             # else consider the marker as FP and delete it. like the BIC algorithm.
    
    if not os.path.exists(out_path):
        os.mkdir(out_path)
    imsave(out_path+"mask"+str(i).zfill(3)+".tif", im_out.astype(np.uint16))   # Fix dpi value for all available results. STAPLE executable requires inputs with same dpi

### Jaccard score computation

In [6]:
import shutil
shutil.rmtree(temp_path)

In [7]:
n_images = len(goldGT_TRA_path)
jaccard_per_time = []
for i in range(n_images):
    
    im_name = imread(out_path+"mask"+str(i).zfill(3)+".tif")
    goldGT_TRA_dir =imread(goldGT_TRA_path[i])
    img_true=np.array(goldGT_TRA_dir).ravel()
    img_pred=np.array(im_name).ravel()
    j = jaccard_score(img_true, img_pred, average=None)
    jaccard_per_time.append(np.mean(j[1:])) # Ignoring the first element, i.e, background label 0 matching
    
    #Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
    # If None, the scores for each class are returned. 
iou = np.mean(np.mean(np.array(jaccard_per_time)))
print("DICE score:", 2*iou/(1+iou))

DICE score: 0.7229560662619186
